## Machine Leaning 실습

In [1]:
# 자동완성
%config Completer.use_jedi = False

### Regression With Python

In [2]:
# 1. 패키지 호출
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

1.2.4
1.19.5


In [3]:
%ls

 01.Regression_with_Python_210602.ipynb
 01SR_Data.csv
 02.Classification_with_Python.ipynb
 02Social_Network_Ads.csv
'03.Classification_with_scikitlearn(Titanic).ipynb'
 03Titanic_dataset.csv
 ML.zip


In [4]:
# 2. 데이터 with pandas DataFrame
df = pd.read_csv('01SR_Data.csv')

In [5]:
# 3-1. 데이터 살펴보기
df.head()
df.tail()

,Country,Age,Year,Salary
5,Germany,40.0,10.0,61000
6,France,37.0,7.0,67000
7,France,44.0,15.0,72000
8,France,48.0,NaN,79000
9,Germany,50.0,21.0,83000


In [6]:
# 3-2. 데이터 정보확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  10 non-null     object 
 1   Age      9 non-null      float64
 2   Year     7 non-null      float64
 3   Salary   10 non-null     int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 448.0+ bytes


In [7]:
# 3-3. 데이터 설명보기
df.describe(include = 'all') # object 열 포함

,Country,Age,Year,Salary
count,10,9.000000,7.000000,10.000000
unique,3,NaN,NaN,NaN
top,France,NaN,NaN,NaN
freq,4,NaN,NaN,NaN
mean,NaN,38.777778,9.142857,63500.000000
std,NaN,7.693793,6.817345,11597.413505
min,NaN,27.000000,2.000000,48000.000000
25%,NaN,35.000000,4.500000,55000.000000
50%,NaN,38.000000,7.000000,61000.000000
75%,NaN,44.000000,12.500000,70750.000000


* AGE은 27세부터 50세의 범위로 평균 38.7세
* YEAR은 결측치 3개 존재
* country는 France, Germany, Spain 존재

In [8]:
# 국가 별 데이터 파악
df.value_counts()

Country  Age   Year  Salary
France   37.0  7.0   67000     1
         44.0  15.0  72000     1
Germany  30.0  2.0   54000     1
         40.0  10.0  61000     1
         50.0  21.0  83000     1
Spain    27.0  3.0   48000     1
dtype: int64

In [9]:
# 4. feature/label 나누기

# 열 이름으로 접근
# feature = df[["Country", "Age", "Year"]]
# lagel = df[["Salary"]]

# idex로 접근
# df.ilof[행, 열] 모든 행 [:,열] 
feature = df.iloc[:,:-1]
label = df.iloc[:,-1]


In [10]:
feature.head()

,Country,Age,Year
0,Spain,27.0,3.0
1,Spain,NaN,6.0
2,Germany,30.0,2.0
3,France,35.0,NaN
4,Spain,38.0,NaN


In [11]:
label.head()

0    48000
1    52000
2    54000
3    58000
4    61000
Name: Salary, dtype: int64

In [12]:
# 5. 비어있는 값 채우기(mean)
# 비어있는 값 확인
df.isnull().sum()

Country    0
Age        1
Year       3
Salary     0
dtype: int64

In [13]:
# 비어있는 값 채우기
from sklearn.impute import SimpleImputer


mean_imputer = SimpleImputer(strategy = 'mean')
mean_imputer.fit(feature.iloc[:, 1:])
feature.iloc[:, 1:] = mean_imputer.transform(feature.iloc[:, 1:])

feature.isnull().sum()

Country    0
Age        0
Year       0
dtype: int64

In [14]:
# 6. One hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), [0])], 
                      remainder = 'passthrough')
feature = ct.fit_transform(feature)

print(feature)

[[ 0.          0.          1.         27.          3.        ]
 [ 0.          0.          1.         38.77777778  6.        ]
 [ 0.          1.          0.         30.          2.        ]
 [ 1.          0.          0.         35.          9.14285714]
 [ 0.          0.          1.         38.          9.14285714]
 [ 0.          1.          0.         40.         10.        ]
 [ 1.          0.          0.         37.          7.        ]
 [ 1.          0.          0.         44.         15.        ]
 [ 1.          0.          0.         48.          9.14285714]
 [ 0.          1.          0.         50.         21.        ]]


In [15]:
# 7. Split Data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size = 0.2, random_state = 123)

print(X_train)
print(y_train)

[[ 1.          0.          0.         44.         15.        ]
 [ 0.          1.          0.         40.         10.        ]
 [ 1.          0.          0.         48.          9.14285714]
 [ 1.          0.          0.         35.          9.14285714]
 [ 0.          0.          1.         38.77777778  6.        ]
 [ 1.          0.          0.         37.          7.        ]
 [ 0.          1.          0.         50.         21.        ]
 [ 0.          1.          0.         30.          2.        ]]
7    72000
5    61000
8    79000
3    58000
1    52000
6    67000
9    83000
2    54000
Name: Salary, dtype: int64


In [16]:
# 8. Train

from sklearn.linear_model import LinearRegression

linear_model = LinearRegression() # y = wx + b(bias, intercept)
linear_model.fit(X_train, y_train) # fit -> trian

LinearRegression()

In [17]:
# 8-1. Train_2

from sklearn.tree import DecisionTreeRegressor
# fit을 동시에 할 수 있음
tree_model = DecisionTreeRegressor().fit(X_train, y_train)


In [18]:
X_test

array([[ 0.        ,  0.        ,  1.        , 38.        ,  9.14285714],
       [ 0.        ,  0.        ,  1.        , 27.        ,  3.        ]])

In [19]:
# 9. Score
y_pred_lr = linear_model.predict(X_test)
print(y_pred_lr)
print(y_test)

[50666.69395455 34855.43405324]
4    61000
0    48000
Name: Salary, dtype: int64


In [20]:
y_pred_tree = tree_model.predict(X_test)
print(y_pred_tree)
print(y_test)

[61000. 52000.]
4    61000
0    48000
Name: Salary, dtype: int64


In [21]:
# 10. Evalute

# 회귀 지표 : mae, mse, R^2 확인
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred_lr)
mse = mean_squared_error(y_test, y_pred_lr)
rmse = mean_squared_error(y_test, y_pred_lr, squared = False) # rmse
r2 = r2_score(y_test, y_pred_lr)

print('LinerRegression_MAE:', mae)
print('LinerRegression_MSE:', mse)
print('LinerRegression_RMSE:', rmse)
print('LinerRegression_R^2:', r2)

LinerRegression_MAE: 11738.93599610473
LinerRegression_MSE: 139778413.8788216
LinerRegression_RMSE: 11822.792135482277
LinerRegression_R^2: -2.3083648255342393


In [23]:
# 트리모델
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred_tree)
mse = mean_squared_error(y_test, y_pred_tree)
rmse = mean_squared_error(y_test, y_pred_tree, squared = False)
r2 = r2_score(y_test, y_pred_tree)

print('DecisionTreeRegressor_MAE:', mae)
print('DecisionTreeRegressor_MSE:', mse)
print('LinerRegression_RMSE:', rmse)
print('DecisionTreeRegressor_R^2:', r2)

DecisionTreeRegressor_MAE: 2000.0
DecisionTreeRegressor_MSE: 8000000.0
LinerRegression_RMSE: 2828.42712474619
DecisionTreeRegressor_R^2: 0.8106508875739645
